In [ ]:
pip install pinecone-client langchain sentence-transformers tqdm pinecone

  Using cached pinecone_client-6.0.0-py3-none-any.whl.metadata (3.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.9/421.9 kB 6.9 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade transformers
!pip install langchain_huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 84.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.50.3
    Uninstalling transformers-4.50.3:
      Successfully uninstalled transformers-4.50.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 823.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.50
    Uninstalling langchain-core-0.3.50:
      Successfully uninstalled langchain-core-0.3.50
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.7
    Uninstalling langchain-text-splitters-0.3.7:
      Successfully uninstalled langchain-text-splitters-0.3.7
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.22
    Uninstalling langchain-0.3.22:
      Successfully uninstalled langchain-0.3.22


In [ ]:
from langchain.vectorstores import Pinecone as LangchainPinecone

In [ ]:
import os
import json
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone, ServerlessSpec
import pinecone
from langchain.docstore.document import Document

In [ ]:
import os

os.environ["PINECONE_API_KEY"] = "your-api-key"
PINECONE_API_KEY = "your-api-key"
PINECONE_ENV = "us-east-1"
INDEX_NAME = "legal-db"

In [ ]:
pc = Pinecone(api_key=PINECONE_API_KEY)

In [ ]:
if INDEX_NAME not in pc.list_indexes().names():
    pc.create_index(
        name=INDEX_NAME,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region=PINECONE_ENV)
    )

In [ ]:
# Connect to index
index = pc.Index(INDEX_NAME)

In [ ]:
# Load Q&A pairs
def load_qa_pairs(json_paths):
    pairs = []
    for path in json_paths:
        with open(path, 'r', encoding='utf-8') as f:
            data = json.load(f)
            pairs.extend(data)
    return pairs

In [ ]:
# Embed and upsert
def upload_to_pinecone(pairs):
    embed_model = SentenceTransformer("all-MiniLM-L6-v2")
    vectors = []
    for i, pair in enumerate(tqdm(pairs)):
        content = f"Q: {pair['question']}\nA: {pair['answer']}"
        vector = embed_model.encode(content).tolist()
        metadata = {
            "question": pair["question"],
            "answer": pair["answer"]
        }
        vectors.append({
            "id": f"qa-{i}",
            "values": vector,
            "metadata": metadata
        })

        # Batch upsert every 100
        if len(vectors) == 100:
            index.upsert(vectors=vectors, namespace="rag-legal")
            vectors = []

    # Final flush
    if vectors:
        index.upsert(vectors=vectors, namespace="rag-legal")

    print(f"✅ Uploaded {len(pairs)} documents.")

In [ ]:
if __name__ == "__main__":
    json_files = ["constitution_qa.json", "crpc_qa.json", "ipc_qa.json"]
    qa_pairs = load_qa_pairs(json_files)
    upload_to_pinecone(qa_pairs)

100%|██████████| 14543/14543 [02:30<00:00, 96.51it/s] 


✅ Uploaded 14543 documents.
